## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1113. Reported Posts (Easy)**

**Table: Actions**

| Column Name   | Type    |
|---------------|---------|
| user_id       | int     |
| post_id       | int     |
| action_date   | date    | 
| action        | enum    |
| extra         | varchar |

This table may have duplicate rows.
The action column is an ENUM (category) type of ('view', 'like', 'reaction', 'comment', 'report', 'share').
The extra column has optional information about the action, such as a reason for the report or a type of reaction.
extra is never NULL.
 
**Write a solution to report the number of posts reported yesterday for each report reason. Assume today is 2019-07-05.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:** 

**Actions table:**

| user_id | post_id | action_date | action | extra  |
|---------|---------|-------------|--------|--------|
| 1       | 1       | 2019-07-01  | view   | null   |
| 1       | 1       | 2019-07-01  | like   | null   |
| 1       | 1       | 2019-07-01  | share  | null   |
| 2       | 4       | 2019-07-04  | view   | null   |
| 2       | 4       | 2019-07-04  | report | spam   |
| 3       | 4       | 2019-07-04  | view   | null   |
| 3       | 4       | 2019-07-04  | report | spam   |
| 4       | 3       | 2019-07-02  | view   | null   |
| 4       | 3       | 2019-07-02  | report | spam   |
| 5       | 2       | 2019-07-04  | view   | null   |
| 5       | 2       | 2019-07-04  | report | racism |
| 5       | 5       | 2019-07-04  | view   | null   |
| 5       | 5       | 2019-07-04  | report | racism |

**Output:** 
| report_reason | report_count |
|---------------|--------------|
| spam          | 1            |
| racism        | 2            |

**Explanation:** Note that we only care about report reasons with non-zero number of reports.

In [0]:
actions_data_1113 = [
    (1, 1, "2019-07-01", "view", None),
    (1, 1, "2019-07-01", "like", None),
    (1, 1, "2019-07-01", "share", None),
    (2, 4, "2019-07-04", "view", None),
    (2, 4, "2019-07-04", "report", "spam"),
    (3, 4, "2019-07-04", "view", None),
    (3, 4, "2019-07-04", "report", "spam"),
    (4, 3, "2019-07-02", "view", None),
    (4, 3, "2019-07-02", "report", "spam"),
    (5, 2, "2019-07-04", "view", None),
    (5, 2, "2019-07-04", "report", "racism"),
    (5, 5, "2019-07-04", "view", None),
    (5, 5, "2019-07-04", "report", "racism"),
]

actions_columns_1113 = ["user_id", "post_id", "action_date", "action", "extra"]
actions_df_1113 = spark.createDataFrame(actions_data_1113, actions_columns_1113)
actions_df_1113.show()

+-------+-------+-----------+------+------+
|user_id|post_id|action_date|action| extra|
+-------+-------+-----------+------+------+
|      1|      1| 2019-07-01|  view|  NULL|
|      1|      1| 2019-07-01|  like|  NULL|
|      1|      1| 2019-07-01| share|  NULL|
|      2|      4| 2019-07-04|  view|  NULL|
|      2|      4| 2019-07-04|report|  spam|
|      3|      4| 2019-07-04|  view|  NULL|
|      3|      4| 2019-07-04|report|  spam|
|      4|      3| 2019-07-02|  view|  NULL|
|      4|      3| 2019-07-02|report|  spam|
|      5|      2| 2019-07-04|  view|  NULL|
|      5|      2| 2019-07-04|report|racism|
|      5|      5| 2019-07-04|  view|  NULL|
|      5|      5| 2019-07-04|report|racism|
+-------+-------+-----------+------+------+



In [0]:
actions_df_1113\
            .filter(
                (col("action") == "report") & 
                (col("action_date") == "2019-07-04"))\
            .groupBy("extra").agg(countDistinct("post_id").alias("report_count")) \
                .withColumnRenamed("extra", "report_reason").show()

+-------------+------------+
|report_reason|report_count|
+-------------+------------+
|       racism|           2|
|         spam|           1|
+-------------+------------+

